In [1]:
!pip install -q BlackBoxAuditing 
!pip install -q aif360

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 5.1 MB/s eta 0:00:0000:01


In [2]:
from sklearn.datasets import fetch_openml
#from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score,precision_score,recall_score
import seaborn as sns
#from fairlearn.metrics import selection_rate
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from aif360.datasets import BinaryLabelDataset
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.metrics import accuracy_score, confusion_matrix
from aif360.metrics import ClassificationMetric
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, Dropout, Input
from tqdm import tqdm




# import numpy as np
# import pandas as pd
# import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
# import os
# import cv2
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split, StratifiedKFold
# from sklearn.metrics import accuracy_score
# import keras
# from keras.layers import *
# from keras.models import *
# from keras.optimizers import Adam
# from aif360.datasets import StandardDataset
# from aif360.metrics import ClassificationMetric

# import warnings
# warnings.simplefilter(action='ignore', category='PerformanceWarning')


In [209]:
df = pd.read_csv('/kaggle/input/70k-job-applicants-data-human-resource/stackoverflow_full.csv').drop("Unnamed: 0",axis=1)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73462 entries, 0 to 73461
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             73462 non-null  object 
 1   Accessibility   73462 non-null  object 
 2   EdLevel         73462 non-null  object 
 3   Employment      73462 non-null  int64  
 4   Gender          73462 non-null  object 
 5   MentalHealth    73462 non-null  object 
 6   MainBranch      73462 non-null  object 
 7   YearsCode       73462 non-null  int64  
 8   YearsCodePro    73462 non-null  int64  
 9   Country         73462 non-null  object 
 10  PreviousSalary  73462 non-null  float64
 11  HaveWorkedWith  73399 non-null  object 
 12  ComputerSkills  73462 non-null  int64  
 13  Employed        73462 non-null  int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 7.8+ MB


,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,51552.0,C++;Python;Git;PostgreSQL,4,0
1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,46482.0,Bash/Shell;HTML/CSS;JavaScript;Node.js;SQL;Typ...,12,1
2,<35,No,Master,1,Man,No,Dev,15,6,Germany,77290.0,C;C++;Java;Perl;Ruby;Git;Ruby on Rails,7,0
3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,46135.0,Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL;Gi...,13,0
4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,160932.0,C++;Python,2,0


In [210]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder

# def preprocess_data(df):
#     # Fill numeric columns with median
#     numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
#     df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())
    
#     # Fill categorical columns with mode
#     categorical_columns = df.select_dtypes(include=['object']).columns
#     df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])
    
#     # Label encode categorical columns
#     le = LabelEncoder()
#     for col in categorical_columns:
#         df[col] = le.fit_transform(df[col])
    
#     return df

# # Example usage:
# df = preprocess_data(df)
# df

In [211]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

def preprocess_data(df):
    """
    Preprocess a pandas DataFrame by:
    1. Filling missing values in numeric columns with median
    2. Filling missing values in categorical columns with mode
    3. Label encoding categorical columns
    4. MinMax scaling numeric columns to range [0,1]
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    
    Returns:
    pandas.DataFrame: Preprocessed DataFrame
    """
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Identify numeric and categorical columns
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.drop("Employed")
    categorical_columns = df.select_dtypes(include=['object']).columns
    
    # Fill numeric columns with median
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())
    
    # Fill categorical columns with mode
    df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])
    
    # Label encode categorical columns
    le = LabelEncoder()
    for col in categorical_columns:
        df[col] = le.fit_transform(df[col])
    
    # MinMax scale numeric columns
    scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    df[categorical_columns] = scaler.fit_transform(df[categorical_columns])
    
    return df

# Example usage:
df = preprocess_data(df)
df

,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,0.0,0.0,0.00,1.0,0.0,0.0,0.0,0.14,0.08,0.853801,0.230139,0.541191,0.037383,0
1,0.0,0.0,1.00,1.0,0.0,0.0,0.0,0.24,0.10,0.824561,0.207505,0.246374,0.112150,1
2,0.0,0.0,0.00,1.0,0.0,0.0,0.0,0.30,0.12,0.315789,0.345042,0.585218,0.065421,0
3,0.0,0.0,1.00,1.0,0.0,0.0,0.0,0.18,0.12,0.157895,0.205956,0.253833,0.121495,0
4,1.0,0.0,0.75,0.0,0.0,0.0,1.0,0.80,0.60,0.789474,0.718445,0.539776,0.018692,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73457,0.0,0.0,1.00,1.0,0.0,0.0,0.0,0.14,0.04,0.315789,0.183291,0.457209,0.121495,1
73458,1.0,0.0,1.00,1.0,0.0,0.0,0.0,0.42,0.32,0.959064,0.513391,0.393661,0.102804,1
73459,0.0,0.0,1.00,1.0,0.0,0.0,0.0,0.08,0.06,0.643275,0.257675,0.842496,0.112150,1
73460,0.0,1.0,1.00,1.0,0.0,1.0,0.0,0.10,0.02,0.959064,0.312497,0.453522,0.140187,1


# Preparing data

# Splitting data

In [212]:
X=df.drop('Employed',axis=1)
y=df['Employed']
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state = 0, stratify=y)

In [213]:
y.value_counts()

Employed
1    39392
0    34070
Name: count, dtype: int64

# Original Model 

In [214]:
from catboost import CatBoostClassifier

In [215]:
model1 = CatBoostClassifier(verbose=0)
model1.fit(X_train, y_train)

In [216]:
y_pred_test = model1.predict(X_test)
y_pred = model1.predict(X)

In [217]:
accuracy_score(y_test.values, y_pred_test)

0.8397195943646635

# Metrics

In [218]:

# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Employed'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = y_pred_test
#np.argmax(y_pred_test).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1357760925765058
Max Average Odds Difference (Ideal value = 0): 0.06533336242213672
Max Equal Opportunity Difference (Ideal value = 0): 0.07639436126433763
Max Theil Index (Ideal value = 0): 0.11742148539096982
Max Generalized Entropy Index (Ideal value = 0): 0.0825105967163194
Max Disparate Impact (Ideal value = 1): 1.3221071132400084


# Mitigation Algorithms

## Preprocessing Algorithms:

### 1- Reweighing

In [219]:

aif360_dataset_train = StandardDataset(df=pd.concat([X_train, y_train], axis=1),
                                           label_name='Employed', favorable_classes=[0],
                                           protected_attribute_names=['Gender'],
                                           privileged_classes=[[1]])
                                          
weights = aif360_dataset_train.instance_weights
preprocessor = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
train_dataset_reweighted = preprocessor.fit_transform(aif360_dataset_train)
#metric_e = BinaryLabelDatasetMetric(train_dataset_reweighted, unprivileged_groups=[{sensitive_attribute: 0}], privileged_groups=[{sensitive_attribute: 1}])

model_weigh = CatBoostClassifier(verbose=0)
model_weigh.fit(train_dataset_reweighted.features,train_dataset_reweighted.labels.ravel() , sample_weight=weights)
#aif360_dataset_train.features  aif360_dataset_train.labels.ravel()  
y_pred_wei = model_weigh.predict(X_test)

In [220]:
accuracy_score(y_test.values, y_pred_wei)

0.8397195943646635

In [221]:
# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
#_X_test = pd.DataFrame(X_test, columns=X.columns)
#_y_test = pd.DataFrame(y_test, columns=['Employed'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset1 = aif360_dataset.copy()
        aif360_pred_dataset1.labels = y_pred_wei
#np.argmax(y_pred_wei).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset1,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1357760925765058
Max Average Odds Difference (Ideal value = 0): 0.06533336242213672
Max Equal Opportunity Difference (Ideal value = 0): 0.07639436126433763
Max Theil Index (Ideal value = 0): 0.11742148539096982
Max Generalized Entropy Index (Ideal value = 0): 0.0825105967163194
Max Disparate Impact (Ideal value = 1): 1.3221071132400084


## 2- Disparate Impact Remover

In [222]:
  
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Employed'])
 

transformed_X_Train = pd.DataFrame(index=X_train.index, columns=X_train.columns)

groups = [
    {'Gender': 0},
    {'Gender': 1},
                  ]
priv_group = groups[0]
unprivileged_groups = [groups[1]]
aif360_dataset = StandardDataset(pd.concat([X_train,y_train],axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

   
DIR = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="Gender")
aif360_dataset_train = DIR.fit_transform(aif360_dataset)   
X_Train_transformed = aif360_dataset_train.features
y_Train_transformed = aif360_dataset_train.labels.ravel()
            


model_DIR = CatBoostClassifier(verbose=0)
model_DIR.fit(X_Train_transformed.astype('float32'), y_Train_transformed.astype('float32'))
y_pred_DIR= model_DIR.predict(X_test)
accuracy_score(y_test.values, y_pred_DIR)

0.7682569931259784

In [223]:
# Initialize variables to store maximum values of each metric

max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Employed'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset2 = aif360_dataset.copy()
        aif360_pred_dataset2.labels = y_pred_DIR
#np.argmax(y_pred_wei).values,axis=1
        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset2,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.33295148915966893
Max Average Odds Difference (Ideal value = 0): 0.27530889615711346
Max Equal Opportunity Difference (Ideal value = 0): 0.33229105628632816
Max Theil Index (Ideal value = 0): 0.25099930647888163
Max Generalized Entropy Index (Ideal value = 0): 0.149850932620175
Max Disparate Impact (Ideal value = 1): 2.347880152759425


# In-processing Algorithms:

## 1- Adversarial Debiasing

In [228]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Employed', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[0]])

# Apply the AdversarialDebiasing algorithm
tf.reset_default_graph()
sess = tf.Session()
adv_debiasing = AdversarialDebiasing(privileged_groups=priv_group,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adv_debiasing',
                                     debias=True,
                                     sess=sess, num_epochs=10)
adv_debiasing.fit(aif360_train)

# Transform the training data
aif360_train_pred = adv_debiasing.predict(aif360_train)
X_Train_transformed = aif360_train_pred.features
y_Train_transformed = aif360_train_pred.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

scores.append(accuracy_score(y_test.values, preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

epoch 0; iter: 0; batch classifier loss: 0.684631; batch adversarial loss: 0.687384
epoch 0; iter: 200; batch classifier loss: 0.616938; batch adversarial loss: 0.572498
epoch 0; iter: 400; batch classifier loss: 0.862514; batch adversarial loss: 0.526232
epoch 1; iter: 0; batch classifier loss: 0.948680; batch adversarial loss: 0.536936
epoch 1; iter: 200; batch classifier loss: 0.987468; batch adversarial loss: 0.522071
epoch 1; iter: 400; batch classifier loss: 1.286402; batch adversarial loss: 0.466958
epoch 2; iter: 0; batch classifier loss: 1.173888; batch adversarial loss: 0.423370
epoch 2; iter: 200; batch classifier loss: 1.443294; batch adversarial loss: 0.381310
epoch 2; iter: 400; batch classifier loss: 1.589118; batch adversarial loss: 0.342635
epoch 3; iter: 0; batch classifier loss: 1.406483; batch adversarial loss: 0.344810
epoch 3; iter: 200; batch classifier loss: 0.551426; batch adversarial loss: 0.332028
epoch 3; iter: 400; batch classifier loss: 0.804939; batch adv

In [229]:
# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf
_X_test = pd.DataFrame(X_test, columns=X.columns)
_y_test = pd.DataFrame(y_test, columns=['Employed'])


for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([_X_test, _y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset2 = aif360_dataset.copy()
        aif360_pred_dataset2.labels =y_pred_4
        #
#np.argmax(y_pred_4.values,axis=1)


        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]

#aif360_pred_dataset.labels = 

        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset2,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())
            #max_di = max(max_di, metric.disparate_impact())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)
#print("Max Disparate Impact (Ideal value = 1):", max_di)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.13021668923166824
Max Average Odds Difference (Ideal value = 0): 0.06883402648049584
Max Equal Opportunity Difference (Ideal value = 0): 0.08062408653425202
Max Theil Index (Ideal value = 0): 0.15614203399291368
Max Generalized Entropy Index (Ideal value = 0): 0.1082135013902698


# Postprocessing Algorithms:

## 1- Equalized Odds Postprocessing

In [230]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import EqOddsPostprocessing,CalibratedEqOddsPostprocessing

from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Employed', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_train, y_train)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds


aif360_test = StandardDataset(pd.concat([X_test, pd.DataFrame(y_test, columns=['Employed'], index=X_test.index)], axis=1),
                                      label_name='Employed', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
aif360_preds = StandardDataset(pd.concat([X_test,pd.DataFrame(y_pred_4, columns=['Employed'], index=X_test.index)], axis=1),
                               label_name='Employed', favorable_classes=[1],
                               protected_attribute_names=['Gender'],
                               privileged_classes=[[1]])

eq_odds = EqOddsPostprocessing(unprivileged_groups=[{'Gender': 1}], privileged_groups=[{'Gender': 0}], seed=42)
eq_odds = eq_odds.fit(aif360_test, aif360_preds)
transformed_preds = eq_odds.predict(aif360_preds).labels



scores.append(accuracy_score(y_test.values, transformed_preds))
print(scores[-1])


print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

0.8150820118423739
Mean: 0.8150820118423739 
STD:  0.0 



In [231]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = transformed_preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.06675881795913258
Max Average Odds Difference (Ideal value = 0): 6.613777625003037e-05
Max Equal Opportunity Difference (Ideal value = 0): 0.00588575027109306
Max Theil Index (Ideal value = 0): 0.13576345308399768
Max Generalized Entropy Index (Ideal value = 0): 0.09562187170846338


# 3- Calibrated EqOdds Postprocessing

In [232]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import EqOddsPostprocessing,CalibratedEqOddsPostprocessing

from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Employed', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_train, y_train)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds


aif360_test = StandardDataset(pd.concat([X_test, pd.DataFrame(y_test, columns=['Employed'], index=X_test.index)], axis=1),
                                      label_name='Employed', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
aif360_preds = StandardDataset(pd.concat([X_test,pd.DataFrame(y_pred_4, columns=['Employed'], index=X_test.index)], axis=1),
                               label_name='Employed', favorable_classes=[1],
                               protected_attribute_names=['Gender'],
                               privileged_classes=[[1]])

eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=[{'Gender': 1}], privileged_groups=[{'Gender': 0}], seed=42)
eq_odds = eq_odds.fit(aif360_test, aif360_preds)
transformed_preds = eq_odds.predict(aif360_preds).labels



scores.append(accuracy_score(y_test.values, transformed_preds))
print(scores[-1])


print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

0.8314163206969305
Mean: 0.8314163206969305 
STD:  0.0 



In [233]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = transformed_preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.15969238555109477
Max Average Odds Difference (Ideal value = 0): 0.09188372187315696
Max Equal Opportunity Difference (Ideal value = 0): 0.10205393354580605
Max Theil Index (Ideal value = 0): 0.11832307375462181
Max Generalized Entropy Index (Ideal value = 0): 0.08487399518939873


## Sequential Models

### RW + AD

In [235]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# for train_index, test_index in StratifiedKFold(n_splits=5).split(X.values, y.values):
#     X_Train, X_Test = X.iloc[train_index, :], X.iloc[test_index, :]
#     y_Train, y_Test = y.iloc[train_index], y.iloc[test_index]

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Employed', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])
weights = aif360_train.instance_weights
preprocessor = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
train_dataset_reweighted = preprocessor.fit_transform(aif360_train)

# Apply the AdversarialDebiasing algorithm
tf.reset_default_graph()
sess = tf.Session()
adv_debiasing = AdversarialDebiasing(privileged_groups=priv_group,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adv_debiasing',
                                     debias=True,
                                     sess=sess, num_epochs=10)
adv_debiasing.fit(train_dataset_reweighted)

# Transform the training data
aif360_train_pred = adv_debiasing.predict(train_dataset_reweighted)
X_Train_transformed = aif360_train_pred.features
y_Train_transformed = aif360_train_pred.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed, sample_weight=weights)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

scores.append(accuracy_score(y_test.values, preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

epoch 0; iter: 0; batch classifier loss: 0.713409; batch adversarial loss: 0.794921
epoch 0; iter: 200; batch classifier loss: 0.803879; batch adversarial loss: 0.682961
epoch 0; iter: 400; batch classifier loss: 0.585713; batch adversarial loss: 0.517648
epoch 1; iter: 0; batch classifier loss: 0.587449; batch adversarial loss: 0.488850
epoch 1; iter: 200; batch classifier loss: 0.727438; batch adversarial loss: 0.476912
epoch 1; iter: 400; batch classifier loss: 0.778513; batch adversarial loss: 0.432839
epoch 2; iter: 0; batch classifier loss: 0.755991; batch adversarial loss: 0.417027
epoch 2; iter: 200; batch classifier loss: 0.752349; batch adversarial loss: 0.353954
epoch 2; iter: 400; batch classifier loss: 0.654327; batch adversarial loss: 0.353945
epoch 3; iter: 0; batch classifier loss: 0.720127; batch adversarial loss: 0.352242
epoch 3; iter: 200; batch classifier loss: 0.737050; batch adversarial loss: 0.289039
epoch 3; iter: 400; batch classifier loss: 0.706672; batch adv

In [236]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.1251832591166519
Max Average Odds Difference (Ideal value = 0): 0.06279773339297466
Max Equal Opportunity Difference (Ideal value = 0): 0.07904383288544048
Max Theil Index (Ideal value = 0): 0.14966173658569906
Max Generalized Entropy Index (Ideal value = 0): 0.10512695888616375


### DIR + AD

In [237]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []


# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Employed', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

DIR = DisparateImpactRemover(repair_level=1.0, sensitive_attribute="Gender")
aif360_train = DIR.fit_transform(aif360_train)   

# Apply the AdversarialDebiasing algorithm
tf.reset_default_graph()
sess = tf.Session()
adv_debiasing = AdversarialDebiasing(privileged_groups=priv_group,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adv_debiasing',
                                     debias=True,
                                     sess=sess, num_epochs=10)
adv_debiasing.fit(aif360_train)

# Transform the training data
aif360_train_pred = adv_debiasing.predict(aif360_train)
X_Train_transformed = aif360_train_pred.features
y_Train_transformed = aif360_train_pred.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

scores.append(accuracy_score(y_test.values, preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

epoch 0; iter: 0; batch classifier loss: 0.693960; batch adversarial loss: 1.001549
epoch 0; iter: 200; batch classifier loss: 1.010743; batch adversarial loss: 0.949096
epoch 0; iter: 400; batch classifier loss: 0.959742; batch adversarial loss: 0.739036
epoch 1; iter: 0; batch classifier loss: 0.766811; batch adversarial loss: 0.629140
epoch 1; iter: 200; batch classifier loss: 0.598583; batch adversarial loss: 0.477906
epoch 1; iter: 400; batch classifier loss: 0.541202; batch adversarial loss: 0.414951
epoch 2; iter: 0; batch classifier loss: 0.513353; batch adversarial loss: 0.422032
epoch 2; iter: 200; batch classifier loss: 0.475297; batch adversarial loss: 0.376773
epoch 2; iter: 400; batch classifier loss: 0.485949; batch adversarial loss: 0.339356
epoch 3; iter: 0; batch classifier loss: 0.526182; batch adversarial loss: 0.290240
epoch 3; iter: 200; batch classifier loss: 0.376974; batch adversarial loss: 0.300114
epoch 3; iter: 400; batch classifier loss: 0.437133; batch adv

In [238]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = y_pred_4

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.22671761400543125
Max Average Odds Difference (Ideal value = 0): 0.1715713284959231
Max Equal Opportunity Difference (Ideal value = 0): 0.25471213621331823
Max Theil Index (Ideal value = 0): 0.2813672778726938
Max Generalized Entropy Index (Ideal value = 0): 0.17219077601680446


### AD + CEOP

In [239]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []


# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Employed', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])

# Apply the AdversarialDebiasing algorithm
tf.reset_default_graph()
sess = tf.Session()
adv_debiasing = AdversarialDebiasing(privileged_groups=priv_group,
                                     unprivileged_groups=unprivileged_groups,
                                     scope_name='adv_debiasing',
                                     debias=True,
                                     sess=sess, num_epochs=10)
adv_debiasing.fit(aif360_train)

# Transform the training data
aif360_train_pred = adv_debiasing.predict(aif360_train)
X_Train_transformed = aif360_train_pred.features
y_Train_transformed = aif360_train_pred.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

aif360_test = StandardDataset(pd.concat([X_test, pd.DataFrame(y_test, columns=['Employed'], index=X_test.index)], axis=1),
                                      label_name='Employed', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
aif360_preds = StandardDataset(pd.concat([X_test,pd.DataFrame(y_pred_4, columns=['Employed'], index=X_test.index)], axis=1),
                               label_name='Employed', favorable_classes=[1],
                               protected_attribute_names=['Gender'],
                               privileged_classes=[[1]])

eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}], seed=42)
eq_odds = eq_odds.fit(aif360_test, aif360_preds)
transformed_preds = eq_odds.predict(aif360_preds).labels

scores.append(accuracy_score(y_test.values, transformed_preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

epoch 0; iter: 0; batch classifier loss: 0.699201; batch adversarial loss: 0.676284
epoch 0; iter: 200; batch classifier loss: 0.899241; batch adversarial loss: 0.571112
epoch 0; iter: 400; batch classifier loss: 0.887539; batch adversarial loss: 0.535505
epoch 1; iter: 0; batch classifier loss: 1.022536; batch adversarial loss: 0.526945
epoch 1; iter: 200; batch classifier loss: 0.953808; batch adversarial loss: 0.469553
epoch 1; iter: 400; batch classifier loss: 0.908127; batch adversarial loss: 0.395175
epoch 2; iter: 0; batch classifier loss: 1.112335; batch adversarial loss: 0.386340
epoch 2; iter: 200; batch classifier loss: 0.886046; batch adversarial loss: 0.397911
epoch 2; iter: 400; batch classifier loss: 0.754691; batch adversarial loss: 0.295576
epoch 3; iter: 0; batch classifier loss: 0.725927; batch adversarial loss: 0.307866
epoch 3; iter: 200; batch classifier loss: 0.630986; batch adversarial loss: 0.277949
epoch 3; iter: 400; batch classifier loss: 0.470495; batch adv

In [240]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = transformed_preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.05905345687997776
Max Average Odds Difference (Ideal value = 0): 0.11237784016004362
Max Equal Opportunity Difference (Ideal value = 0): 0.13613771611110648
Max Theil Index (Ideal value = 0): 0.1959009536744456
Max Generalized Entropy Index (Ideal value = 0): 0.13743858736413064


## RW + CEOP

In [241]:
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Function to convert dataframe to AIF360 StandardDataset
def convert_to_dataset(df, label_name, favorable_classes, protected_attribute_names, privileged_classes):
    return StandardDataset(df,
                           label_name=label_name,
                           favorable_classes=favorable_classes,
                           protected_attribute_names=protected_attribute_names,
                           privileged_classes=privileged_classes)

y_pred_4 = pd.Series(0, index=X.index)
groups = [
    {'Gender': 0},
    {'Gender': 1},
]
priv_group = groups[0]
unprivileged_groups = [groups[1]]

scores = []

# for train_index, test_index in StratifiedKFold(n_splits=5).split(X.values, y.values):
#     X_Train, X_Test = X.iloc[train_index, :], X.iloc[test_index, :]
#     y_Train, y_Test = y.iloc[train_index], y.iloc[test_index]

# Create a combined DataFrame for X_Train and y_Train
combined_train = pd.concat([X_train, y_train], axis=1)

# Convert to AIF360 format
aif360_train = convert_to_dataset(combined_train,
                                  label_name='Employed', favorable_classes=[1],
                                  protected_attribute_names=['Gender'],
                                  privileged_classes=[[1]])
weights = aif360_train.instance_weights
preprocessor = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
train_dataset_reweighted = preprocessor.fit_transform(aif360_train)
X_Train_transformed = train_dataset_reweighted.features
y_Train_transformed = train_dataset_reweighted.labels.ravel()

# Train your model using the debiased training data
model = CatBoostClassifier(verbose=0)
model.fit(X_Train_transformed, y_Train_transformed, sample_weight=weights)

# Get predictions from your model
preds = model.predict(X_test.values)
y_pred_4 = preds

aif360_test = StandardDataset(pd.concat([X_test, pd.DataFrame(y_test, columns=['Employed'], index=X_test.index)], axis=1),
                                      label_name='Employed', favorable_classes=[1],
                                      protected_attribute_names=['Gender'],
                                      privileged_classes=[[1]])
aif360_preds = StandardDataset(pd.concat([X_test,pd.DataFrame(y_pred_4, columns=['Employed'], index=X_test.index)], axis=1),
                               label_name='Employed', favorable_classes=[1],
                               protected_attribute_names=['Gender'],
                               privileged_classes=[[1]])

eq_odds = CalibratedEqOddsPostprocessing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}], seed=42)
eq_odds = eq_odds.fit(aif360_test, aif360_preds)
transformed_preds = eq_odds.predict(aif360_preds).labels



scores.append(accuracy_score(y_test.values, transformed_preds))
print(scores[-1])

sess.close()

print("Mean:", np.mean(scores), "\nSTD: ", np.std(scores), '\n')

0.8314843803171579
Mean: 0.8314843803171579 
STD:  0.0 



In [242]:
import pandas as pd
import numpy as np


# Initialize variables to store maximum values of each metric
max_spd = max_aod = max_eod = max_theil = max_gei = max_di = -np.inf

for age_cls in [0,1]:
    for priv_cls in [0,1]:
        # Convert to AIF360 format
        aif360_dataset = StandardDataset(df=pd.concat([X_test, y_test], axis=1),
                                         label_name='Employed', favorable_classes=[age_cls],
                                         protected_attribute_names=['Gender'],
                                         privileged_classes=[[priv_cls]])

        # Create a dataset for the predictions
        aif360_pred_dataset = aif360_dataset.copy()
        aif360_pred_dataset.labels = transformed_preds

        import numpy as np

        # Define the groups
        groups = [
            {'Gender': 0},
            {'Gender': 1}
        ]



        # Iterate through the groups
        for i, privileged_group in enumerate(groups):
            # Define unprivileged groups as all other groups
            unprivileged_groups = groups[:i] + groups[i+1:]

            # Calculate metrics
            metric = ClassificationMetric(aif360_dataset, aif360_pred_dataset,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=[privileged_group])

            # Update maximum values
            max_spd = max(max_spd, metric.statistical_parity_difference())
            max_aod = max(max_aod, metric.average_odds_difference())
            max_eod = max(max_eod, metric.equal_opportunity_difference())
            max_theil = max(max_theil, metric.theil_index())
            max_gei = max(max_gei, metric.generalized_entropy_index())

# Print maximum values
print("Gender Bias Identification:")
print("Max Statistical Parity Difference (Ideal value = 0):", max_spd)
print("Max Average Odds Difference (Ideal value = 0):", max_aod)
print("Max Equal Opportunity Difference (Ideal value = 0):", max_eod)
print("Max Theil Index (Ideal value = 0):", max_theil)
print("Max Generalized Entropy Index (Ideal value = 0):", max_gei)

Gender Bias Identification:
Max Statistical Parity Difference (Ideal value = 0): 0.15819761724018294
Max Average Odds Difference (Ideal value = 0): 0.09017723723151876
Max Equal Opportunity Difference (Ideal value = 0): 0.09864096426252966
Max Theil Index (Ideal value = 0): 0.1183045311335472
Max Generalized Entropy Index (Ideal value = 0): 0.08485107738549914
